In [ ]:
pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install fastT5

# ONNX Model Generation

In [ ]:
from fastT5 import export_and_get_onnx_model, generate_onnx_representation, quantize
from transformers import T5Config, AutoTokenizer

In [ ]:
trained_model = "prithivida/grammar_error_correcter_v1"

onnx_model_paths = generate_onnx_representation(trained_model)

quant_onnx_paths = quantize(onnx_model_paths)

tokenizer_onnx = AutoTokenizer.from_pretrained(trained_model)
config = T5Config.from_pretrained(trained_model)


Exporting to onnx... |################################| 3/3
Quantizing... |################################| 3/3


In [ ]:
tokenizer_onnx.save_pretrained('models/')
config.save_pretrained('models/')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cp -r models '/content/gdrive/My Drive/Grammar_Correction'

# ONNX Runtime

In [ ]:
from fastT5 import get_onnx_model, get_onnx_runtime_sessions, OnnxT5
from transformers import AutoTokenizer

def correct(input):
  trained_model_path = "/content/drive/MyDrive/Grammar_Correction/models"

  encoder_path = trained_model_path+"/grammar_error_correcter_v1-encoder-quantized.onnx"
  decoder_path = trained_model_path+"/grammar_error_correcter_v1-decoder-quantized.onnx"
  init_decoder_path = trained_model_path+"/grammar_error_correcter_v1-init-decoder-quantized.onnx"

  model_paths = encoder_path,decoder_path,init_decoder_path
  model_sessions = get_onnx_runtime_sessions(model_paths)
  model = OnnxT5(trained_model_path,model_sessions)

  tokenizer = AutoTokenizer.from_pretrained(trained_model_path)

  prompt = 'gec:' + input
  token = tokenizer(prompt, return_tensors='pt')
  tokens = model.generate(input_ids=token['input_ids'],
               attention_mask=token['attention_mask'],
               num_beams=2)

  output = tokenizer.decode(tokens.squeeze(), skip_special_tokens=True)
  return output

In [ ]:
print(correct("I name is Sandeep"))

My name is Sandeep.
